In [134]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import pandas_bokeh
# import finplot as fplt

import polygon_rest_api as papi
import polygon_backfill as pb
# import ray_backfill as rb
import polygon_s3 as ps3
import bar_samples as bs
import bar_labels as bl

# import filters as ft
# import market_cluster as mc

import matplotlib as mpl
mpl.rcParams['figure.figsize'] = [20, 7]
from matplotlib import style
style.use('fivethirtyeight')

# import ipdb
# ipdb.set_trace(context=10)

# df.style.background_gradient(cmap='coolwarm', axis=None, vmin=-1, vmax=1)  # 'Blues', 'coolwarm', 'RdBu_r' & 'BrBG' are other good diverging colormaps
# pd.set_option('precision', 2)

# get local market daily data
# mdf = pd.read_parquet('data/mdf.parquet')

# .plot_bokeh(sizing_mode="scale_height")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [24]:
symbol='GLD'
# symbol = 'market'
date = '2020-10-05'
start_date = '2020-01-01' 
end_date = '2020-02-20'
symbols=['SPY', 'QQQ']
tick_type='trades'
# tick_type = None
result_path = '/Users/bobcolner/QuantClarity/pandas-polygon/data'

In [182]:
ps3.get_symbol_dates(symbol='SPY', tick_type=tick_type)

['2020-01-02',
 '2020-01-03',
 '2020-01-06',
 '2020-01-07',
 '2020-01-08',
 '2020-01-09',
 '2020-01-10',
 '2020-01-13',
 '2020-01-14',
 '2020-01-15',
 '2020-01-16',
 '2020-01-17',
 '2020-01-21',
 '2020-01-22',
 '2020-01-23',
 '2020-01-24',
 '2020-01-27',
 '2020-01-28',
 '2020-01-29',
 '2020-01-30',
 '2020-01-31',
 '2020-02-03',
 '2020-02-04',
 '2020-02-05',
 '2020-02-06',
 '2020-02-07',
 '2020-02-10',
 '2020-02-11',
 '2020-02-12',
 '2020-02-13',
 '2020-02-14',
 '2020-02-18',
 '2020-02-19',
 '2020-02-20',
 '2020-02-21']

In [171]:
ps3.find_symbol_storage_used(symbol, tick_type)

141387596

In [173]:
ps3.get_symbol_dates(symbol='SPY', tick_type=tick_type)

['2020-01-02',
 '2020-01-03',
 '2020-01-06',
 '2020-01-07',
 '2020-01-08',
 '2020-01-09',
 '2020-01-10',
 '2020-01-13',
 '2020-01-14',
 '2020-01-15',
 '2020-01-16',
 '2020-01-17',
 '2020-01-21',
 '2020-01-22']

In [ ]:
;# sample ticks -> bars
renko_size = 0.2
# renko_reversal = 2
thresh = {
    # GT thresholds
    'renko_size': renko_size,
    'renko_reveral_multiple': 2,
    'duration_sec': 60*30,
    'tick_imbalance': 150,
    # LT thresholds 
    'min_duration_sec': 15,
    'min_tick_count': 30,
}
# ticks_df['epoch'] = ticks_df.date_time.astype(int)
tdf = ticks_df.loc[(ticks_df.date_time.dt.hour >= 13) & (ticks_df.date_time.dt.hour <= 22)]
bars, _ = bs.build_bars(ticks_df=tdf, thresh=thresh)
len(bars)

bars_df = pd.DataFrame(bars).set_index('close_at')
# bars_df = pd.DataFrame(bars)

In [ ]:
# range / price = HFT profit potential
(df.price.max() - df.price.min()) / df.price.mean()

In [ ]:
# label bars
label_horizon_mins = 30
risk_level = 0.2
# reward_base = 1
reward_ratios = np.arange(3, 20, .5)

lbars = bl.label_bars(bars, ticks_df, risk_level, label_horizon_mins, reward_ratios)

lbars_df = pd.DataFrame(lbars)

lbars_unq = bl.get_concurrent_stats(lbars_df)
print(lbars_unq['grand_avg_unq'])

In [ ]:
lbars_df[['price_wmean', 'label_rrr']].plot_bokeh(sizing_mode="scale_height")
# 17, 31

In [ ]:
lbars_df[['price_wmean', 'label_rrr']].plot(subplots=True, figsize=[17,12])

In [ ]:
label_prices, price_end_at = bl.get_label_ticks(tdf, bars[31]['close_at'], 30)